In [1]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193638 sha256=18d9b2c6de5468a6ccdda3aa97264842b8a19bcc2d2f0d62db951ab822342d6d
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader 
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

Realizo la ingesta de los csv que obtuve del EDA 

In [4]:
df_scores = pd.read_csv ("/content/drive/MyDrive/Data/ML/score.csv")
df_titles = pd.read_csv ("/content/drive/MyDrive/Data/ML/titles.csv")

In [6]:
df_scores.shape

(11013875, 3)

In [5]:
reader = Reader()

In [7]:
# Como podemos observar el dataset tiene una cantidad muy grande de filas, por lo que limitamos su número a 100 mil

N_filas= 100000
data = Dataset.load_from_df(df_scores[['userId', 'id', 'score']][:N_filas], reader)

In [8]:
# Separamos nuestros datos

trainset, testset = train_test_split(data, test_size=.25)

In [9]:
# Utilizo el modelo de Singular Value Decomposition

model = SVD()

In [10]:
# Entrenamos el modelo

model.fit(trainset)

In [11]:
# Predecimos

predictions = model.test(testset)

In [12]:
predictions[1]

Prediction(uid=14738, iid='as192', r_ui=3.0, est=3.7355801945360687, details={'was_impossible': False})

In [13]:
# Hacemos una predicción al azar para usuario y película

model.predict(611,"as1")

Prediction(uid=611, iid='as1', r_ui=None, est=3.1222905887438306, details={'was_impossible': False})

In [14]:
# Elijo un usuario para hacerle una recomendación

usuario = 543
rating = 4   # Tomamos películas a las que haya calificado con 4 o 5 estrellas
df_user = df_scores[(df_scores['userId'] == usuario) & (df_scores['score'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user = df_user.join(df_titles.set_index('id'), on='id', how='inner')


In [15]:
df_user.head ()

,userId,score,id,title
0,543,5.0,as1,the grand seduction
1,543,5.0,as31,yu-gi-oh! 5d's
2,543,4.0,as166,waiting for you
3,543,4.0,as168,waiting for anya
4,543,4.0,as201,valentine road


In [16]:
recomendaciones_usuario = df_titles.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(4499, 2)


,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [17]:
# Extraigo las películas que ya ha visto

usuario_vistas = df_scores[df_scores['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

(553, 3)


,userId,score,id
0,543,5.0,as1
5359,543,3.0,as12
7732,543,3.0,as17
14378,543,5.0,as31
71849,543,1.0,as151


In [18]:
# Recomendamos

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['id'].apply(lambda x: model.predict(usuario, x).est)

In [19]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

        id                                            title  Estimate_Score
107  as108                                           whoop!        4.219992
24    as25                                         zandalee        4.147130
56    as57                                  yancy derringer        4.111413
131  as132                what the constitution means to me        4.101268
0      as1                              the grand seduction        4.045365
186  as187  vietnam: secret negotiations that ended the war        3.936465
15    as16                                       summer '03        3.914696
73    as74                          words on bathroom walls        3.867987
61    as62                                       wwii in hd        3.866134
143  as144                             we'll take manhattan        3.864173


In [20]:
# Evaluación del Modelo
accuracy.rmse(predictions)

RMSE: 1.0349


1.034875622268197